In [9]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import transforms
from utils.dataset_generator import CartaDataset
from utils.models import SimpleCNN

In [10]:
df = pd.read_csv('db\\dados_cartas_yugioh.csv', encoding='utf-8')
df

,id,nome,raca,caminho_imagem
0,11714098,"30,000-Year White Turtle",Aqua,imagens\Aqua\11714098.jpg
1,18318842,Abyss Soldier,Aqua,imagens\Aqua\18318842.jpg
2,88409165,Abyss Warrior,Aqua,imagens\Aqua\88409165.jpg
3,17080584,"Abyssrhine, the Atlantean Spirit",Aqua,imagens\Aqua\17080584.jpg
4,46358784,Advanced Crystal Beast Emerald Tortoise,Aqua,imagens\Aqua\46358784.jpg
...,...,...,...,...
8523,85065943,Saint Azamina,Illusion,imagens\Illusion\85065943.jpg
8524,1528054,Silhouhatte Rabbit,Illusion,imagens\Illusion\1528054.jpg
8525,74150658,Talons of Shurilane,Illusion,imagens\Illusion\74150658.jpg
8526,9275482,UFOLight,Illusion,imagens\Illusion\9275482.jpg


In [11]:
le = LabelEncoder()
df['raca_codificada'] = le.fit_transform(df['raca'])

# (Opcional) Ver os mapeamentos
classe_to_index = dict(zip(le.classes_, le.transform(le.classes_)))
print("Mapeamento das classes:", classe_to_index)

Mapeamento das classes: {'Aqua': np.int64(0), 'Beast': np.int64(1), 'Beast-Warrior': np.int64(2), 'Dinosaur': np.int64(3), 'Divine-Beast': np.int64(4), 'Dragon': np.int64(5), 'Fairy': np.int64(6), 'Fiend': np.int64(7), 'Fish': np.int64(8), 'Illusion': np.int64(9), 'Insect': np.int64(10), 'Machine': np.int64(11), 'Plant': np.int64(12), 'Psychic': np.int64(13), 'Pyro': np.int64(14), 'Reptile': np.int64(15), 'Rock': np.int64(16), 'Sea Serpent': np.int64(17), 'Spellcaster': np.int64(18), 'Thunder': np.int64(19), 'Warrior': np.int64(20), 'Winged Beast': np.int64(21), 'Wyrm': np.int64(22), 'Zombie': np.int64(23)}


In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [13]:
dataset = CartaDataset(df, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 6. Exemplo de iteração pelo dataloader
for imagens, labels in dataloader:
    print("Batch de imagens:", imagens.shape)
    print("Batch de labels:", labels)
    break


Batch de imagens: torch.Size([32, 3, 224, 224])
Batch de labels: tensor([ 5,  7, 13,  1,  3,  2, 21, 20,  6,  4,  2,  1, 10,  1,  5, 20,  6, 21,
         0, 18, 11,  1, 11, 20, 20,  7,  0,  5, 20,  5, 23, 23])


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=len(le.classes_))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    print(f"Época [{epoch+1}/{num_epochs}]")
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataset)
    print(f"Perda da época {epoch+1}: {epoch_loss:.4f}", end="\n\n")
    

Época [1/10]


UnidentifiedImageError: cannot identify image file 'imagens\\Dragon\\662855.jpg'

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Acurácia no dataset: {100 * correct / total:.2f}%")

Acurácia no dataset: 100.00%
